In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from lazypredict.Supervised import LazyClassifier
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    classification_report, accuracy_score,
    precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve
)

In [2]:
df = pd.read_csv("../data/processed/final_data.csv")
df.head()

,date,total_precipitation,rain_only,soil_moisture_0_7cm,soil_moisture_7_28cm,soil_moisture_28_100cm,soil_moisture_100_255cm,surface_pressure,humidity,soil_moisture_avg,rain_sum_6h,hydro_saturation_index,rain_lag_1h,soil_lag_1h,rain_lag_3h,soil_lag_3h,rain_lag_6h,soil_lag_6h,target_alert_level
0,2024-01-01 06:00:00,0.00,0.00,0.49,0.48,0.34,0.40,1008.01,97.63,0.43,0.00,0.00,0.00,0.43,0.00,0.43,0.00,0.43,0
1,2024-01-01 07:00:00,0.00,0.00,0.49,0.48,0.34,0.40,1008.82,93.09,0.43,0.00,0.00,0.00,0.43,0.00,0.43,0.00,0.43,0
2,2024-01-01 08:00:00,0.00,0.00,0.49,0.48,0.34,0.40,1008.84,84.49,0.43,0.00,0.00,0.00,0.43,0.00,0.43,0.00,0.43,0
3,2024-01-01 09:00:00,0.10,0.10,0.49,0.48,0.34,0.40,1009.34,74.26,0.43,0.10,0.05,0.00,0.43,0.00,0.43,0.00,0.43,0
4,2024-01-01 10:00:00,0.10,0.10,0.49,0.48,0.34,0.40,1009.25,70.29,0.43,0.20,0.05,0.10,0.43,0.00,0.43,0.00,0.43,0


In [3]:
# 1. Tentukan Fitur (X) dan Target (y)
# Kita hapus kolom non-numerik seperti 'date' dan kolom target itu sendiri
X = df.drop(columns=['target_alert_level', 'date'])
y = df['target_alert_level']

In [4]:
# Membagi data menjadi set pelatihan dan set pengujian
split_index = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

# Tampilkan bentuk set pelatihan dan set uji untuk memastikan split
print("Training set shape:")
print(f"X_train={X_train.shape}")
print(f"y_train={y_train.shape} \n")
print("Test set shape:")
print(f"X_train={X_test.shape}")
print(f"y_train={y_test.shape}")

Training set shape:
X_train=(14030, 17)
y_train=(14030,) 

Test set shape:
X_train=(3508, 17)
y_train=(3508,)


In [5]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
def evaluate_model(y_true, y_pred, model_name="Model", train_accuracy=None, target_names=None):
    """
    Menampilkan evaluasi lengkap: Confusion Matrix, classification report, dan metrik evaluasi.

    Parameters:
    - y_true: array-like, label sebenarnya
    - y_pred: array-like, label hasil prediksi
    - model_name: str, nama model (default: "Model")
    - train_accuracy: float, akurasi pada data latih (opsional)
    - target_names: list of str, nama label untuk classification report
    """
    # Metode evaluasi
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    print(f"🛑 EVALUASI MODEL {model_name.upper()}")
    print("=" * 60)
    print(f"Accuracy : {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall   : {recall:.2f}")
    print(f"F1 Score : {f1:.2f}\n")

    print(f"🛑 CLASSIFICATION REPORT")
    print("=" * 60)
    print(classification_report(y_true, y_pred, target_names=target_names))
    print("=" * 60)
    if train_accuracy is not None:
        print(f"*{model_name.upper()} - Akurasi pada data latih: {train_accuracy:.4f}")
    print(f"*{model_name.upper()} - Akurasi pada data uji: {accuracy:.4f}")
    print("=" * 60)
    print("\n")
    print("Visualisasi:")

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots(figsize=(6, 5))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_names)
    disp.plot(ax=ax, cmap='Blues', colorbar=True)
    plt.title(f"Confusion Matrix - {model_name}")
    plt.tight_layout()
    plt.show()

In [ ]:
clf = LazyClassifier()
models, predicts = clf.fit(X_train, X_test, y_train, y_test)

print(models.sort_values(by="Accuracy", ascending=False))

2026/01/22 13:09:43 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2026/01/22 13:09:44 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2026/01/22 13:09:44 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


  0%|          | 0/32 [00:00<?, ?it/s]